## Running on a GPU

To speed things up further Theano can [make use of your GPU](http://deeplearning.net/software/theano/tutorial/using_gpu.html). Theano relies on Nvidia's CUDA platform, so yo need a [CUDA-enabled graphics card](http://docs.nvidia.com/cuda/cuda-getting-started-guide-for-mac-os-x/#cuda-enabled-gpu). If you don't have one (for example if you're on a Macbook with an integrated GPU like me) you could spin up a [GPU-optimized Amazon EC2 instance](https://aws.amazon.com/ec2/instance-types/#gpu) and try things there. To use the GPU with Theano you must have the CUDA toolkit installed and [set a few environment variables](http://deeplearning.net/software/theano/install.html#gpu-linux). The [repository readme](https://github.com/dennybritz/nn-theano) contains instructions on how to setup CUDA on an EC2 isntance.

While any Theano code will run just fine with the GPU enabled you probably want to spend time optimizing your code get optimal performance. Running code that isn't optimized for a GPU may actually be slower on a GPU than a CPU due to excessive data transfers.

Here are  some things you must consider when writing Theano code for a GPU:

- The default floating point data type is `float64`, but in order to use the GPU you must use `float32`. There are several ways to do this in Theano. You can either use the [fulled typed tensor constructors](http://deeplearning.net/software/theano/library/tensor/basic.html#all-fully-typed-constructors) like `T.fvector` or set  `theano.config.floatX` to `float32` and use the [simple tensor constructors](http://deeplearning.net/software/theano/library/tensor/basic.html#constructors-with-optional-dtype) like `T.vector`.
- Shared variables must be initialized with values of type `float32` in order to be stored on the GPU. In other words, when creating a shared variable from a numpy array you must initialize the array with the `dtype=float32` argument, or cast it using the `astype` function. See the [numpy data type documentation](http://docs.scipy.org/doc/numpy/user/basics.types.html) for more details.
- Be very careful with data transfers to and from the GPU. Ideally you want all your data on the GPU. Ways to achieve this include initializing commonly used data as shared variables with a `float32` data type, and to [avoid copying return values](http://deeplearning.net/software/theano/tutorial/using_gpu.html#returning-a-handle-to-device-allocated-data) back to the host using the `gpu_from_host` function. Theano includes [profiling capabilities](http://deeplearning.net/software/theano/tutorial/profiling.html) that allow you to debug how much time you are spending on data transfers.

With all that in mind, let's modify our code to run well on a GPU. I'll highlight the most important changes we here, but the full code is available [here](https://github.com/dennybritz/nn-theano) or at the bottom of the post.

In [ ]:
import numpy as np
import sklearn
import sklearn.datasets
import theano
import theano.tensor as T
import time

# Use float32 as the default float data type
theano.config.floatX = 'float32'

First, to see a significant speedup we should increase the size of our data so that the computation time will be dominated by matrix multiplication and other mathematical operations (as opposed to looping in Python code and assigning values). 

In [ ]:
train_X, train_y = sklearn.datasets.make_moons(5000, noise=0.20)
train_X = train_X.astype(np.float32)
train_y = train_y.astype(np.int32)
nn_hdim = 1000

Next, we change our scalar values to `float32` to ensure that our calculations stay within that type. If we multiplied a `float32` value by a `float64` value the return value would be a `float64`.

In [ ]:
epsilon = np.float32(0.01) # learning rate for gradient descent
reg_lambda = np.float32(0.01) # regularization strength 

To force the storage of our data on the GPU we use shared variables of type `float32`. Because we are constantly using the input data `X` and `y` to perform gradient updates we store those on the GPU as well:

In [ ]:
# These float32 values will be initialized on the GPU
X = theano.shared(train_X.astype('float32'), name='X')
y = theano.shared(train_y_onehot.astype('float32'), name='y')
W1 = theano.shared(np.random.randn(nn_input_dim, nn_hdim).astype('float32'), name='W1')
b1 = theano.shared(np.zeros(nn_hdim).astype('float32'), name='b1')
W2 = theano.shared(np.random.randn(nn_hdim, nn_output_dim).astype('float32'), name='W2')
b2 = theano.shared(np.zeros(nn_output_dim).astype('float32'), name='b2')

We remove the inputs from our functions since the X and y values are stored as shared variables (on the GPU) now. We no longer need to pass them to the function calls.

In [ ]:
gradient_step = theano.function(
    inputs=[],
    updates=((W2, W2 - epsilon * dW2),
             (W1, W1 - epsilon * dW1),
             (b2, b2 - epsilon * db2),
             (b1, b1 - epsilon * db1)))

# Does a gradient step and updates the values of our paremeters
gradient_step()

### Running and Evaluation

To use a GPU you must run the code with the `THEANO_FLAGS=device=gpu,floatX=float32` environment variable set. I tested the GPU-optimized code on a `g2.2xlarge` AWS EC2 instance. Running one `gradient_step()` on the CPU took around **250ms**. With the GPU enabled it merely took **7.5ms**. That's a 30x speedup, and if our dataset or parameter space were larger we would probably see an even more significant speedup.



In [ ]:
%timeit gradient_step()

### Full GPU-optimzied code

In [ ]:
# Generate a dataset
np.random.seed(0)
train_X, train_y = sklearn.datasets.make_moons(5000, noise=0.20)
train_y_onehot = np.eye(2)[train_y]

# Size definitions
num_examples = len(train_X) # training set size
nn_input_dim = 2 # input layer dimensionality
nn_output_dim = 2 # output layer dimensionality
nn_hdim = 1000 # hiden layer dimensionality

# Gradient descent parameters (I picked these by hand)
epsilon = np.float32(0.01) # learning rate for gradient descent
reg_lambda = np.float32(0.01) # regularization strength 

# GPU NOTE: Conversion to float32 to store them on the GPU!
X = theano.shared(train_X.astype('float32')) # initialized on the GPU
y = theano.shared(train_y_onehot.astype('float32'))

# GPU NOTE: Conversion to float32 to store them on the GPU!
W1 = theano.shared(np.random.randn(nn_input_dim, nn_hdim).astype('float32'), name='W1')
b1 = theano.shared(np.zeros(nn_hdim).astype('float32'), name='b1')
W2 = theano.shared(np.random.randn(nn_hdim, nn_output_dim).astype('float32'), name='W2')
b2 = theano.shared(np.zeros(nn_output_dim).astype('float32'), name='b2')

# Forward propagation
z1 = X.dot(W1) + b1
a1 = T.tanh(z1)
z2 = a1.dot(W2) + b2
y_hat = T.nnet.softmax(z2)

# The regularization term (optional)
loss_reg = 1./num_examples * reg_lambda/2 * (T.sum(T.sqr(W1)) + T.sum(T.sqr(W2))) 
# the loss function we want to optimize
loss = T.nnet.categorical_crossentropy(y_hat, y).mean() + loss_reg
# Returns a class prediction
prediction = T.argmax(y_hat, axis=1)

# Gradients
dW2 = T.grad(loss, W2)
db2 = T.grad(loss, b2)
dW1 = T.grad(loss, W1)
db1 = T.grad(loss, b1)

# Note that we removed the input values because we will always use the same shared variable
# GPU NOTE: Removed the input values to avoid copying data to the GPU.
forward_prop = theano.function([], y_hat)
calculate_loss = theano.function([], loss)
predict = theano.function([], prediction)

# GPU NOTE: Removed the input values to avoid copying data to the GPU.
gradient_step = theano.function(
    [],
    # profile=True,
    updates=((W2, W2 - epsilon * dW2),
             (W1, W1 - epsilon * dW1),
             (b2, b2 - epsilon * db2),
             (b1, b1 - epsilon * db1)))

def build_model(num_passes=20000, print_loss=False):
    # Re-Initialize the parameters to random values. We need to learn these.
    np.random.seed(0)
    # GPU NOTE: Conversion to float32 to store them on the GPU!
    W1.set_value((np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim)).astype('float32'))
    b1.set_value(np.zeros(nn_hdim).astype('float32'))
    W2.set_value((np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim)).astype('float32'))
    b2.set_value(np.zeros(nn_output_dim).astype('float32'))
    
    # Gradient descent. For each batch...
    for i in xrange(0, num_passes):
        # This will update our parameters W2, b2, W1 and b1!
        gradient_step()
        
        # Optionally print the loss.
        # This is expensive because it uses the whole dataset, so we don't want to do it too often.
        if print_loss and i % 1000 == 0:
            print "Loss after iteration %i: %f" %(i, calculate_loss())


# Profiling
# theano.config.profile = True
# theano.config.profile_memory = True
# gradient_step()
# theano.printing.debugprint(gradient_step) 
# print gradient_step.profile.summary()
            
%timeit gradient_step()